Определять надо параметры:

    Type
    Finish1
    Treatment
    Hanging Method
    Shade Color
    Shade Shape
    Tier
    Style

Product title - можно использовать в помощь и какие-то атрибуты, возможно, определять по нему, а не по изображению.

In [ ]:
from __future__ import division, unicode_literals

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(3)

In [ ]:
train = pd.read_csv('product_attributes_update.csv', encoding='utf-8', 
                    dialect='excel', lineterminator='\n')#updated?
train.tail(30)

In [ ]:
train.describe(include=['O'])

In [ ]:
len(train)

In [ ]:
excess_columns=set(["ProductID","EID","PID","VID","Department","Product Title","Category"])
train_researsh_columns=set(train.columns)-excess_columns
for ColumnName in ['Hanging Method']:# train_researsh_columns: 
    train[ColumnName].value_counts().plot(kind="bar")
    plt.show()

In [ ]:
#old_train = train
def delete_rare_records(treshold,column_name):
    global train
    e = train[column_name].value_counts()
    return train[train[column_name].isin(e[e>treshold].index)]
Column=delete_rare_records(treshold=5,column_name='Tier')['Tier'] 
Column.groupby(Column).count().plot(kind="bar")
plt.show()
#print(delete_rare_records(treshold=5,column_name='Tier').columns)

In [ ]:
 def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data
 def df2features(df):
    return np.array([
        df['Hanging Method'],
    ]).transpose()

x_train_list=[]
y_train_list=[]

one_hot=pd.get_dummies(train['Hanging Method'])

for row in train.itertuples():
    if pd.notnull(row[11]):
        try:
            img = load_image('..\\ml_150x150_m\\'+row[1]+'.jpg').reshape(150,150,1)
        except IOError:
            continue
        x_train_list.append(img)
        y_train_list.append(one_hot.loc[row[0],:].values)

In [ ]:
x_train=np.array(x_train_list)
y_train=np.array(y_train_list)

In [ ]:
len(y_train)==len(x_train)
len(x_train)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras import backend as K

In [ ]:
K.image_data_format()

In [ ]:
x_validation=x_train[10000:]
y_validation=y_train[10000:]
x_train=x_train[:10000]
y_train=y_train[:10000]

In [ ]:
epochs = 10
batch_size = 32

input_shape = (128, 128, 1)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
metrics=['accuracy'])

In [ ]:
#from keras.utils import plot_model
#plot_model(model, to_file='model.png')
model.summary()

как мы будем тренировать

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True)

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_validation,y_validation),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs)

#model.save('first_try.h5')
#model=load_model(filepath)